In [1]:
import geopandas as gpd
import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import requests

from pysal.viz.splot.mapping import vba_choropleth
from pysal.model.spreg import ols
# from pysal.lib.weights.contiguity import Queen
# from pysal.lib.weights.distance import DistanceBand

/mnt/c/Users/isaac/Dropbox/2019Spring/SpatialComputing/spatial-project/env/lib/python3.6/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# Build the query to get census data

variables = [
    'B01002_001E',  # Median Age
    'B06011_001E',  # Median Income
    'B25088_003E',  # Median SMOC (Selected Monthly Owner Costs)
    # Employment Status
    'B23025_003E',  # Total Civilian Labor Force
    'B23025_005E',  # Total Unemployed
    # Means of Transportation to Work
    'B08101_001E',  # Total
    'B08101_033E',  # Total Walked
    # Educational Attainment
    'B06009_001E',  # Total
    'B06009_002E',  # Total Less Than High School
    'B06009_003E',  # Total High School
    'B06009_004E',  # Total College
    'B06009_005E',  # Total Bachelor
    'B06009_006E',  # Total Graduate
    # Income
    'B19001_001E',  # Total
    'B19001_002E',  # < 10k
    'B19001_003E',  # 10k-15k
    'B19001_004E',  # 15k-25k
    'B19001_005E',  # 25k-35k
    'B19001_006E',  # 35k-50k
    'B19001_007E',  # 50k-75k
    'B19001_008E',  # 75k-100k
    'B19001_009E',  # 100k-150k
    'B19001_010E',  # 150k-200k
    'B19001_011E',  # > 200k
    # Race
    'B02001_001E',  # Total
    'B02001_002E',  # White
    'B02001_003E',  # Black
    'B02001_004E',  # American Indian
    'B02001_005E',  # Asian
    'B02001_006E',  # Pacific Islander
    'B02001_007E',  # Other
    'B02001_008E',  # Two Or More
]

payload = {'get': ','.join(variables), 'for': 'zip code tabulation area:*'}
r = requests.get('https://api.census.gov/data/2017/acs/acs5', params=payload)

In [3]:
# Use pandas to get the census data

census_df = pd.read_json(r.url)
census_df = census_df.rename(columns=census_df.iloc[0])
census_df = census_df.drop(0)

In [4]:
# Calulate new variables, filter Chicago/Seattle zip codes, join Chicago data with Niche grades

def simpson_index(total, cols):
    s = sum([col*(col-1.0) for col in cols])
    return 1.0 - (s/(total * (total - 1.0)))

def transform(df):
    df = df.rename(index = str, columns = {'zip code tabulation area': 'zip',
                                           'B01002_001E': 'median_age',
                                           'B06011_001E': 'median_income',
                                           'B25088_003E': 'median_smoc'})
    df = df.astype('float64')
    df = df.astype({'zip': 'int'})
    df = df.replace(-666666666.0, np.nan)
    
    df['unemployment_rate'] = df.B23025_005E / df.B23025_003E
    df['walked_pct'] = df.B08101_033E / df.B08101_001E
    # df['less_than_high_school_pct'] = df.B06009_002E / df.B06009_001E
    # df['high_school_pct'] = df.B06009_003E / df.B06009_001E
    # df['college_pct'] = df.B06009_004E / df.B06009_001E
    # df['bachelor_pct'] = df.B06009_005E / df.B06009_001E
    # df['graduate_pct'] = df.B06009_006E / df.B06009_001E
    df['higher_education_rate'] = (df.B06009_005E + df.B06009_006E)  / df.B06009_001E
    df['race_simpson_index'] = df.apply(lambda row: simpson_index(row['B02001_001E'], [row['B02001_002E'],
                                                                                       row['B02001_003E'],
                                                                                       row['B02001_004E'],
                                                                                       row['B02001_005E'],
                                                                                       row['B02001_006E'],
                                                                                       row['B02001_007E'],
                                                                                       row['B02001_008E']]),
                                        axis=1)
    df['income_simpson_index'] = df.apply(lambda row: simpson_index(row['B19001_001E'], [row['B19001_002E'],
                                                                                         row['B19001_003E'],
                                                                                         row['B19001_004E'],
                                                                                         row['B19001_005E'],
                                                                                         row['B19001_006E'],
                                                                                         row['B19001_007E'],
                                                                                         row['B19001_008E'],
                                                                                         row['B19001_009E'],
                                                                                         row['B19001_010E'],
                                                                                         row['B19001_011E']]),
                                          axis=1)
    return df

# Chicago
chicago_zip_codes = [60601,60602,60603,60604,60605,60606,60607,60608,60609,60610,60611,60612,60613,60614,60615,60616,60617,60618,60619,60620,60621,60622,60623,60624,60625,60626,60628,60629,60630,60631,60632,60633,60634,60636,60637,60638,60639,60640,60641,60642,60643,60644,60645,60646,60647,60649,60651,60652,60653,60654,60655,60656,60657,60659,60660,60661,60664,60666,60668,60669,60670,60673,60674,60675,60677,60678,60680,60681,60682,60684,60685,60686,60687,60688,60689,60690,60691,60693,60694,60695,60696,60697,60699,60701,60706,60707,60803,60804,60805,60827]

chicago_df = census_df.loc[census_df['zip code tabulation area'].astype('int32').isin(chicago_zip_codes)]

chicago_df = transform(chicago_df)

# Seattle

seattle_zip_codes = [98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109, 98110, 98111, 98112, 98113, 98114, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98124, 98125, 98126, 98127, 98129, 98131, 98133, 98134, 98136, 98138, 98139, 98141, 98144, 98145, 98146, 98148, 98154, 98155, 98158, 98160, 98161, 98164, 98165, 98166, 98168, 98170, 98174, 98175, 98177, 98178, 98181, 98185, 98188, 98190, 98191, 98194, 98195, 98198, 98199]

seattle_df = census_df.loc[census_df['zip code tabulation area'].astype('int32').isin(seattle_zip_codes)]

seattle_df = transform(seattle_df)

# Add Grades
chicago_grades_df = pd.read_csv('data/chicago_grades.csv')
chicago_grades_df = chicago_grades_df.rename(index = str, columns = {'Zip Code': 'zip'})

chicago_df = chicago_df.merge(chicago_grades_df, on='zip')

seattle_df

/mnt/c/Users/isaac/Dropbox/2019Spring/SpatialComputing/spatial-project/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


,median_age,median_income,median_smoc,B23025_003E,B23025_005E,B08101_001E,B08101_033E,B06009_001E,B06009_002E,B06009_003E,...,B02001_005E,B02001_006E,B02001_007E,B02001_008E,zip,unemployment_rate,walked_pct,higher_education_rate,race_simpson_index,income_simpson_index
32339,38.9,50792.0,1044.0,8415.0,419.0,7900.0,3335.0,10704.0,367.0,1248.0,...,1779.0,56.0,287.0,696.0,98101,0.049792,0.422152,0.599496,0.477890,0.970215
32340,33.3,54126.0,808.0,19469.0,506.0,18707.0,3827.0,20456.0,338.0,717.0,...,2734.0,62.0,290.0,1723.0,98102,0.025990,0.204576,0.742081,0.363502,0.984827
32341,33.4,48066.0,728.0,36318.0,1222.0,34644.0,1895.0,39685.0,880.0,2267.0,...,3975.0,77.0,506.0,3329.0,98103,0.033647,0.054699,0.738793,0.297083,0.988393
32342,40.3,24511.0,805.0,7466.0,519.0,6907.0,2819.0,12155.0,2031.0,2033.0,...,3676.0,48.0,353.0,853.0,98104,0.069515,0.408137,0.406828,0.649048,0.944100
32343,23.6,19320.0,862.0,25767.0,2112.0,22910.0,4037.0,21623.0,473.0,1074.0,...,10925.0,181.0,684.0,3234.0,98105,0.081965,0.176211,0.751607,0.507011,0.953796
32344,36.1,33965.0,511.0,15521.0,1232.0,13850.0,391.0,18237.0,2301.0,3033.0,...,4357.0,464.0,1700.0,3273.0,98106,0.079376,0.028231,0.367111,0.697595,0.975406
32345,35.0,52504.0,761.0,17328.0,670.0,16367.0,1115.0,20298.0,418.0,1393.0,...,1600.0,0.0,413.0,1596.0,98107,0.038666,0.068125,0.724899,0.293415,0.985701
32346,35.6,26834.0,558.0,13573.0,1241.0,12094.0,478.0,16467.0,3670.0,3357.0,...,8966.0,203.0,920.0,1842.0,98108,0.091432,0.039524,0.318941,0.723897,0.968777
32347,32.0,60184.0,1000.0,20142.0,833.0,18973.0,4506.0,21973.0,425.0,1122.0,...,4257.0,87.0,111.0,1999.0,98109,0.041356,0.237495,0.745551,0.442974,0.988933
32348,47.7,52072.0,805.0,11229.0,369.0,10659.0,517.0,16839.0,316.0,1101.0,...,803.0,22.0,295.0,1048.0,98110,0.032861,0.048504,0.691728,0.192573,0.992033


In [12]:
# Read Chicago zip codes shape file, join with Chicago data

chicago_gdf = gpd.read_file('data/chicago/chicago.shp')
chicago_gdf = chicago_gdf.astype({'zip': 'int'})
chicago_gdf = chicago_gdf.merge(chicago_df, on="zip")
chicago_gdf = chicago_gdf.drop_duplicates('zip')
chicago_gdf = chicago_gdf.dropna()
chicago_gdf.index = chicago_gdf['zip']
chicago_gdf

,objectid,shape_area,shape_len,zip,geometry,median_age,median_income,median_smoc,B23025_003E,B23025_005E,...,Housing,Nightlife,Good for Families,Diversity,Jobs,Weather,Cost of Living,Health & Fitness,Outdoor Activities,Commute
zip,,,,,,,,,,,,,,,,,,,,,
60647,33.0,1.060523e+08,42720.044406,60647,POLYGON ((-87.67762151065281 41.91775780106298...,31.6,35334.0,803.0,54467.0,2630.0,...,4,12,9,11,7,7,6,11,10,10
60639,34.0,1.274761e+08,48103.782721,60639,"POLYGON ((-87.72683253163021 41.9226462671259,...",32.2,21292.0,666.0,44980.0,4273.0,...,2,11,6,9,4,7,6,9,10,8
60622,36.0,7.085383e+07,42527.989679,60622,"POLYGON ((-87.6670686895295 41.88885188496992,...",31.7,49268.0,982.0,37828.0,1633.0,...,5,12,9,10,8,7,6,12,10,10
60651,37.0,9.903962e+07,47970.140153,60651,POLYGON ((-87.70655631674127 41.89555340699404...,33.0,19819.0,601.0,27909.0,3974.0,...,3,9,7,11,3,7,7,8,10,8
60611,38.0,2.350606e+07,34689.350631,60611,POLYGON ((-87.61401200402452 41.89353997745442...,40.0,72100.0,1499.0,20433.0,719.0,...,4,12,8,9,9,7,5,12,10,12
60638,39.0,1.661663e+08,67710.646739,60638,"POLYGON ((-87.74346534021549 41.8187595760259,...",37.7,31213.0,531.0,30515.0,3059.0,...,7,10,8,11,6,7,8,9,10,8
60652,40.0,1.283098e+08,48187.949880,60652,POLYGON ((-87.68304800178444 41.75740742192918...,35.7,29754.0,496.0,22433.0,2924.0,...,6,8,8,11,5,7,8,8,10,7
60626,1.0,4.917058e+07,33983.913306,60626,POLYGON ((-87.66420656495005 42.02126289858394...,33.9,24674.0,733.0,29655.0,2187.0,...,4,12,8,11,5,7,7,10,10,10
60615,8.0,6.656545e+07,38321.313769,60615,POLYGON ((-87.58103206762146 41.80421129714134...,33.9,29612.0,857.0,21586.0,2203.0,...,4,12,9,11,5,7,6,11,10,11


In [10]:
# Read Seattle zip codes shape file, join with Seattle data

seattle_gdf = gpd.read_file('data/seattle/seattle.shp')
seattle_gdf = seattle_gdf.rename(index = str, columns = {'ZIP': 'zip'})
seattle_gdf = seattle_gdf.astype({'zip': 'int'})
seattle_gdf = seattle_gdf.merge(seattle_df, on="zip")
seattle_gdf = seattle_gdf.drop_duplicates('zip')
seattle_gdf = seattle_gdf.dropna()
seattle_gdf.index = seattle_gdf['zip']
seattle_gdf

,OBJECTID,zip,ZIPCODE,COUNTY,SHAPE_Leng,SHAPE_Area,geometry,median_age,median_income,median_smoc,...,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,unemployment_rate,walked_pct,higher_education_rate,race_simpson_index,income_simpson_index
zip,,,,,,,,,,,,,,,,,,,,,
98101,59,98101,98101,033,34632.584473,2.458772e+07,POLYGON ((-122.3300154794659 47.61843728467148...,38.9,50792.0,1044.0,...,185.0,1779.0,56.0,287.0,696.0,0.049792,0.422152,0.599496,0.477890,0.970215
98102,60,98102,98102,033,37488.780105,4.322189e+07,POLYGON ((-122.3159301696644 47.64951306373531...,33.3,54126.0,808.0,...,45.0,2734.0,62.0,290.0,1723.0,0.025990,0.204576,0.742081,0.363502,0.984827
98103,61,98103,98103,033,71421.212504,1.440083e+08,POLYGON ((-122.3284983624493 47.70111708599151...,33.4,48066.0,728.0,...,155.0,3975.0,77.0,506.0,3329.0,0.033647,0.054699,0.738793,0.297083,0.988393
98104,62,98104,98104,033,43354.179793,3.880558e+07,POLYGON ((-122.3222046332107 47.61204519175887...,40.3,24511.0,805.0,...,285.0,3676.0,48.0,353.0,853.0,0.069515,0.408137,0.406828,0.649048,0.944100
98105,63,98105,98105,033,82483.975807,1.504377e+08,"POLYGON ((-122.309777115396 47.67381195228112,...",23.6,19320.0,862.0,...,224.0,10925.0,181.0,684.0,3234.0,0.081965,0.176211,0.751607,0.507011,0.953796
98106,64,98106,98106,033,87139.546583,1.531829e+08,POLYGON ((-122.3599178331702 47.58242203548352...,36.1,33965.0,511.0,...,195.0,4357.0,464.0,1700.0,3273.0,0.079376,0.028231,0.367111,0.697595,0.975406
98107,65,98107,98107,033,48012.037213,6.711620e+07,POLYGON ((-122.4058534704423 47.67581982127474...,35.0,52504.0,761.0,...,77.0,1600.0,0.0,413.0,1596.0,0.038666,0.068125,0.724899,0.293415,0.985701
98108,66,98108,98108,033,84808.362510,2.150580e+08,POLYGON ((-122.3017459653655 47.57192457736458...,35.6,26834.0,558.0,...,172.0,8966.0,203.0,920.0,1842.0,0.091432,0.039524,0.318941,0.723897,0.968777
98109,67,98109,98109,033,40749.032457,6.635577e+07,POLYGON ((-122.3524308777338 47.64890274094839...,32.0,60184.0,1000.0,...,90.0,4257.0,87.0,111.0,1999.0,0.041356,0.237495,0.745551,0.442974,0.988933


In [26]:
# Write joined shape files to output
gpd.GeoDataFrame(chicago_gdf).to_file('chicago_joined.shp')

In [13]:
# Run a regression for Diversity

def run_ols(gdf, ind_var, dep_vars, dataset):
    y = gdf[[ind_var]].values
    X = gdf[dep_vars].values
    res = ols.OLS(y, X, name_y=ind_var, name_x=dep_vars, name_ds=dataset)
    return res

ind_var = 'Diversity'
dep_vars = [
    'median_age',
    'median_income',
    'median_smoc',
    'unemployment_rate',
    'walked_pct',
    'higher_education_rate',
    'race_simpson_index',
    'income_simpson_index'
    ]

chicago_model = run_ols(chicago_gdf, ind_var, dep_vars, 'chicago')
print(chicago_model.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     chicago
Weights matrix      :        None
Dependent Variable  :   Diversity                Number of Observations:          55
Mean dependent var  :      9.4727                Number of Variables   :           9
S.D. dependent var  :      1.8842                Degrees of Freedom    :          46
R-squared           :      0.8705
Adjusted R-squared  :      0.8480
Sum squared residual:      24.825                F-statistic           :     38.6544
Sigma-square        :       0.540                Prob(F-statistic)     :   6.667e-18
S.E. of regression  :       0.735                Log likelihood        :     -56.166
Sigma-square ML     :       0.451                Akaike info criterion :     130.331
S.E of regression ML:      0.6718                Schwarz criterion     :     148.397

-----------------------------------------------------------------------------

In [20]:
# Predict Seattle grades from Chicago regression model

constant = chicago_model.betas[0][0]
seattle_res = [constant] * len(seattle_df)
seattle_res = np.add(seattle_res, chicago_model.betas[1:].T.dot(seattle_df[dep_vars].values.T)[0])
seattle_res

array([10.3616068 ,  9.56843663,  9.58569777, 12.26152564, 11.51965105,
       12.01065356,  9.1749021 , 12.37965775,  9.71138045,  9.71397582,
        9.59559183,  9.91033541,  9.30101684,  9.16370878, 12.78069659,
        9.71731614,  9.24489084, 11.30166781, 11.26312861, 10.75988785,
       11.41725269, 11.54754703,  9.88979643, 12.70502795, 11.77904036,
       11.26997749,         nan, 11.10414499,         nan,         nan,
       10.82790842, 11.98247765,         nan, 10.72580016, 12.79787151,
       12.06491662,         nan, 11.88643687,  9.27744387])

In [21]:
# View Seattle predictions as grades
def to_grade(n):
    if n > 12:
        return 'A+'
    elif n > 11:
        return 'A'
    elif n > 10:
        return 'A-'
    elif n > 9:
        return 'B+'
    elif n > 8:
        return 'B'
    elif n > 7:
        return 'B-'
    elif n > 6:
        return 'C+'
    elif n > 5:
        return 'C'
    elif n > 4:
        return 'C-'
    elif n > 3:
        return 'D+'
    elif n > 2:
        return 'D'
    elif n > 1:
        return 'D-'
    elif n > 0:
        return 'F'
    

seattle_grades_df = seattle_df[['zip']]
seattle_grades_df['diversity_pred'] = np.asarray([to_grade(n) for n in seattle_res])
seattle_grades_df

/mnt/c/Users/isaac/Dropbox/2019Spring/SpatialComputing/spatial-project/env/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,zip,diversity_pred
32339,98101,A-
32340,98102,B+
32341,98103,B+
32342,98104,A+
32343,98105,A
32344,98106,A+
32345,98107,B+
32346,98108,A+
32347,98109,B+
32348,98110,B+


In [22]:
# Run regressions for multiple variables.

ind_vars = ["Housing", "Diversity", "Jobs", "Cost of Living", "Commute"]
dep_vars = [
    'median_age',
    'median_income',
    'median_smoc',
    'unemployment_rate',
    'walked_pct',
    'higher_education_rate',
    'race_simpson_index',
    'income_simpson_index'
    ]

seattle_grades_pred_df = seattle_df[['zip']]
# seattle_grades_pred_df.index = seattle_grades_pred_df['zip']

# Train Chicago model and test on Seattle data for each independent variable
for ind_var in ind_vars:
    chicago_model = run_ols(chicago_gdf, ind_var, dep_vars, 'chicago')
    constant = chicago_model.betas[0][0]
    seattle_res = [constant] * len(seattle_df)
    seattle_res = np.add(seattle_res, chicago_model.betas[1:].T.dot(seattle_df[dep_vars].values.T)[0])
    col = ind_var + " Prediction"
    seattle_grades_pred_df[col] = np.asarray(seattle_res)
#     seattle_grades_pred_df[col] = np.asarray([to_grade(n) for n in seattle_res])
    
seattle_grades_pred_df = seattle_grades_pred_df.dropna()
seattle_grades_pred_df = seattle_grades_pred_df.sort_values('zip')
seattle_grades_pred_df = seattle_grades_pred_df[seattle_grades_pred_df.zip != 98164] # bc 98164 isn't in seattle_grades_df
seattle_grades_pred_df

/mnt/c/Users/isaac/Dropbox/2019Spring/SpatialComputing/spatial-project/env/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,zip,Housing Prediction,Diversity Prediction,Jobs Prediction,Cost of Living Prediction,Commute Prediction
32339,98101,4.523114,10.361607,6.971762,6.644781,10.934480
32340,98102,6.901467,9.568437,9.207760,6.762135,10.406133
32341,98103,7.187066,9.585698,9.395036,6.684328,9.752874
32342,98104,2.386110,12.261526,3.877745,6.638823,10.812295
32343,98105,3.091989,11.519651,5.589894,5.443115,11.468474
32344,98106,5.874068,12.010654,6.460822,7.419593,9.040550
32345,98107,7.072432,9.174902,9.273462,6.658208,9.751376
32346,98108,4.802050,12.379658,5.491572,7.189518,9.099463
32347,98109,6.471225,9.711380,9.204675,6.515505,10.709652
32348,98110,7.216922,9.713976,9.436601,6.863297,9.026160


In [23]:
# Load scraped Seattle grades

seattle_grades_df = pd.read_csv('data/seattle_grades.csv')
seattle_grades_df = seattle_grades_df.rename(index = str, columns = {'Zip Code': 'zip'})
seattle_grades_df = seattle_grades_df[['zip'] + ind_vars]

# seattle_grades_df.index = seattle_grades_df.zip

seattle_grades_df = seattle_grades_df.sort_values('zip')

In [24]:
# Calculate errors for Seattle predictions

res = np.abs(np.subtract(seattle_grades_df[ind_vars], seattle_grades_pred_df[[s + ' Prediction' for s in ind_vars]]))
print('average error:', np.average(res))

for ind_var in ind_vars:
    res = np.abs(np.subtract(seattle_grades_df[[ind_var]], seattle_grades_pred_df[[ind_var + ' Prediction']]))
    print('average ' + ind_var + ' error:', np.average(res))

average error: 1.1945662404461497
average Housing error: 0.9356140514200149
average Diversity error: 0.8457631828746521
average Jobs error: 0.7293716339665811
average Cost of Living error: 2.519945295433789
average Commute error: 0.9421370385357126


In [19]:
chicago_df[['zip'] + dep_vars].tail()

,zip,median_age,median_income,median_smoc,unemployment_rate,walked_pct,higher_education_rate,race_simpson_index,income_simpson_index
51,60656,41.4,33720.0,640.0,0.056079,0.011344,0.382639,0.274753,0.972547
52,60657,31.2,56048.0,1101.0,0.029657,0.049420,0.816783,0.243040,0.985688
53,60659,34.1,25203.0,758.0,0.074881,0.060201,0.368062,0.616550,0.965295
54,60660,37.7,27112.0,763.0,0.061992,0.064571,0.516454,0.502660,0.959400
55,60661,31.3,80943.0,1115.0,0.040559,0.505156,0.896141,0.529107,0.987993
